In [ ]:
# Import statements

In [ ]:
def read_queries():
    pass

In [ ]:
def read_history(): 
    pass

In [ ]:
def read_docs(): 
    pass

In [ ]:
def read_cached_idf(): 
    pass

In [ ]:
def write_output(results): 
    pass

In [ ]:
def update_history(history, top_k_sections, query): 
    pass

In [ ]:
def write_history(history): 
    pass

In [ ]:
def calculate_idf(all_docs): 
    pass

In [ ]:
def cache_idf(idf): 
    pass

In [ ]:
def segment_docs(relevant_docs, keywords): 
    pass

In [ ]:
def select_relevant_docs(municipalities, time_window, all_docs): 
    pass

In [ ]:
def score_doc_section(doc_section, keywords, idf):
    pass

In [ ]:
def select_top_k(doc_sections, doc_scores, k, history): 
    pass

In [ ]:
def update_results(results, top_k_sections, query): 
    pass

In [ ]:
def run_queries(use_cached_idf = False): 
    queries = read_queries()
    all_docs = read_docs()
    if use_cached_idf:
        idf = read_cached_idf()
    else:
        idf = calculate_idf(all_docs)
        cache_idf(idf)
    history = read_history()
    
    results = []
    
    for query in queries: 
        user_id = query["user_id"]
        query_id = query["query_id"]
        keywords = query["keywords"]
        time_window = query["time_window"]
        municipalities = query["municipalities"]
        relevant_docs = select_relevant_docs(municipalities, time_window, all_docs)
        doc_sections = segment_docs(relevant_docs, keywords)
        doc_sections_scores = score_doc_section(doc_section, keywords, idf)
        top_k_sections = select_top_k(doc_sections, doc_scores, k, history)
        results = update_results(results, top_k_sections, query)
        history = update_history(history, top_k_sections, query)
        
    write_output(results)
    write_history(history)
            

    